### Web Intelligence - Exercise 10

In this exercise, we will explore several prominent techniques for learning word representations. First, we will implement a matrix decomposition-based word embedding model, **Latent Semantic Analysis (LSA)**. These models leverage matrix factorization to capture semantic relationships in text, providing an introduction to the underlying principles of word representation.

Next, we will implement a neural-based approach, the **SkipGram** model of the **Word2Vec** framework. The SkipGram architecture is designed to predict surrounding context words given a target word, effectively learning word representations that capture meaningful relationships between words in a corpus.

We will work on the [CMU Book Summary](https://www.cs.cmu.edu/~dbamman/booksummaries.html) dataset that consists of plot summaries for $16,559$ books extracted from Wikipedia. Throughout the exercise, we will analyze and compare these methods, discussing their advantages, limitations, and practical applications in natural language processing tasks. 

**Question 1.** In this exercise, we will implement the Latent Semantic Analysis (LSA).

In [1]:
import sys
print(sys.executable)

C:\Software\python\Web Intelligence\web-intelligence-course-sw7\venv\Scripts\python.exe


In [2]:
import os.path
import nltk

# Set custom NLTK data directory
project_dir = os.path.dirname(os.path.abspath('exercise10.ipynb'))  # Get current script directory
nltk_data_dir = os.path.join(project_dir, "nltk_data")    # Define 'nltk_data' directory in your project

# Ensure the directory exists
os.makedirs(nltk_data_dir, exist_ok=True)

# Add the directory to NLTK data path
nltk.data.path.append(nltk_data_dir)

# Download resources to the custom path
nltk.download('punkt', download_dir=nltk_data_dir)
nltk.download('stopwords', download_dir=nltk_data_dir)
nltk.download('wordnet', download_dir=nltk_data_dir)
nltk.download('omw-1.4', download_dir=nltk_data_dir)  # Optional: For multilingual support


from tqdm.notebook import tqdm
# For loading the dataset
from datasets import load_dataset

# Preprocessing packages
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer



[nltk_data] Downloading package punkt to C:\Software\python\Web
[nltk_data]     Intelligence\web-intelligence-course-
[nltk_data]     sw7\Jupyter\lecture_10\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Software\python\Web
[nltk_data]     Intelligence\web-intelligence-course-
[nltk_data]     sw7\Jupyter\lecture_10\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Software\python\Web
[nltk_data]     Intelligence\web-intelligence-course-
[nltk_data]     sw7\Jupyter\lecture_10\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Software\python\Web
[nltk_data]     Intelligence\web-intelligence-course-
[nltk_data]     sw7\Jupyter\lecture_10\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Load the dataset and preprocess the book summaries.

In [3]:
# Load the CMU Book Summary Corpus dataset and get all the summaries.
ld = load_dataset("textminr/cmu-book-summaries")['train']
summaries = [data['summary'] for data in ld]

In [4]:
# Complete the following preprocessing steps:
stop_words = set(stopwords.words("english"))

def preprocess_texts(texts, stop_words):
    
    # Initialize the lemmatizer
    lemmatizer = WordNetLemmatizer()
    
    cleaned_texts = []
    for text in tqdm(texts, desc="Pre-processing text"):
        
        # 1. Lowercase the text
        text = text.lower()
        
        # 3. Remove punctuation and special characters &&
        text = re.sub(r'[^\w\s]', '', text)
        
        # 4. Tokenize the text
        words = word_tokenize(text)
        
        # 5. Remove stopwords
        words = [word for word in words if word not in stop_words]
        
        # 6. Lemmatize the tokens 
        words = [lemmatizer.lemmatize(word) for word in words]  # Use verb part-of-speech
        
        # Join tokens back into a single string
        cleaned_texts.append(" ".join(words))
        
    return cleaned_texts

# Run the preprocessing function on the summaries
corpus = preprocess_texts(summaries, stop_words=stop_words)

Pre-processing text:   0%|          | 0/16559 [00:00<?, ?it/s]

In [5]:

# You can compare the original and preprocessed summaries to see the difference.
print(f"Original: {summaries[0][:100]}")
print(f"Preprocessed: {corpus[0][:100]}")

Original:  Old Major, the old boar on the Manor Farm, calls the animals on the farm for a meeting, where he co
Preprocessed: old major old boar manor farm call animal farm meeting compare human parasite teach animal revolutio


Construct the Document-Term Matrix

In [6]:

# Note: you can only consider the top 'N' words ordered by term frequency across the corpus.
from sklearn.feature_extraction.text import CountVectorizer

vocab_size = 10000

# Create a CountVectorizer for the DTM
count_vectorizer = CountVectorizer(stop_words="english", max_features=vocab_size)

# Fit and transform the corpus to create the DTM
dt_matrix = count_vectorizer.fit_transform(corpus)

# Extract the feature names (top N words)
terms = count_vectorizer.get_feature_names_out()

# Display the first few terms
print("\nTerms:\n", terms[:10])





Terms:
 ['10' '100' '1000' '10000' '100000' '11' '12' '12yearold' '13' '13yearold']


Construct the Term Frequency-Inverse Document Frequency (TF-IDF) Weighting

In [44]:
from sklearn.feature_extraction.text import TfidfTransformer

# Initialize the transformer
tfidf_transformer = TfidfTransformer()

# Fit and transform the document-term matrix to the TF-IDF matrix
tfidf_matrix = tfidf_transformer.fit_transform(dt_matrix)

# Display the first few terms
print("\nTF-IDF Matrix:\n", tfidf_matrix[:5])




TF-IDF Matrix:
   (0, 133)	0.04150027829700491
  (0, 163)	0.01787838408697166
  (0, 166)	0.024047851437112504
  (0, 169)	0.021486533255623004
  (0, 205)	0.019968958385918494
  (0, 210)	0.02540244616021007
  (0, 214)	0.023890674032071246
  (0, 248)	0.023424717904722572
  (0, 250)	0.028157402173279493
  (0, 352)	0.09253021954306415
  (0, 388)	0.020200623927263895
  (0, 397)	0.027587918903752555
  (0, 409)	0.02486030023038575
  (0, 482)	0.5900330892276815
  (0, 499)	0.03844616591751648
  (0, 548)	0.013860535224677348
  (0, 590)	0.015843323840120098
  (0, 598)	0.019011204347495193
  (0, 660)	0.021661851751714506
  (0, 695)	0.021978249273469835
  (0, 728)	0.012483440929445712
  (0, 733)	0.03216625739596574
  (0, 814)	0.010648110333267907
  (0, 924)	0.01343425136825808
  (0, 939)	0.042125905444246735
  :	:
  (4, 8816)	0.03207509032705052
  (4, 8819)	0.04728338622713203
  (4, 8839)	0.046573697498296214
  (4, 8849)	0.0378884534874158
  (4, 8860)	0.03687043879351056
  (4, 8980)	0.0464055856439

Perfom Singular Value Decomposition (SVD) and Dimensionality Reduction

In [ ]:
import numpy as np
from scipy import linalg

k= 32

U, S, Vh = linalg.svd(tfidf_matrix, k=k)

# Print the singular values
print("\nTop k- singular values:\n", S)

# Define the word embeddings
# Note that the columns of SVh.T define the word embeddings, i.e. rows of the VhS
# SVh.T is also equal to U^TX so the rows of (X^TU) also correspond the same word embeddings.
word_embeddings = np.dot(np.diag(S), Vh).T


Generate Word Embeddings and Analyze Similarity Between Words

In [2]:
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import seaborn as sns

words_to_explore= [
    'king', 'queen', 'woman', 'man', 'man', 'woman', 'boy', 'daughter', 
    #'dog', 'cat', 'horse', 'elephant', 'fish', 'bird', 'lizard', 'snake',
    #'doctor', 'nurse', 'scientist', 'teacher', 'engineer', 'artist', 'musician', 'writer',
    #'denmark', 'sweden', 'france', 'germany', 'spain', 'italy', 'portugal', 'turkey',
]




In [ ]:
# Analyzing similarity between words
from sklearn.metrics.pairwise import cosine_similarity

def cosine_similarity_terms(word1, word2, word_embeddings_df):
    
    return 

# Find top 5 similar words for a given word
def top_similar_words(word, word_embeddings_df, top_n=5):

    return 

word_embeddings_df = pd.DataFrame(word_embeddings, index=terms)

word1, word2 = "king", "queen"
# Test cosine similarity function
print(f"Cosine similarity between '{word1}' and '{word2}': {cosine_similarity_terms(word1, word2, word_embeddings_df)}")
word = "king"
print(f"Top 5 similar words to '{word}': {top_similar_words(word, word_embeddings_df, top_n=5)}")

Compare Book Summaries

In [ ]:
# Define the book summary vectors
doc_embedding = U

# Compute cosine similarity between documents
doc_similarities = cosine_similarity(doc_embedding)
doc_similarities_df = pd.DataFrame(
    doc_similarities, columns=[f"Summary {i+1}" for i in range(len(corpus))], index=[f"Summary {i+1}" for i in range(len(corpus))])

#print("\nSummary Similarities (Cosine Similarity):\n", doc_similarities_df)

**Question 2.** In this exercise, you will explore pre-trained word embeddings on the [Google News dataset](https://code.google.com/archive/p/word2vec/), and you will familiarize yourself with the **Gensim** package for handling the word embeddings. You will also investigate the analogy relationships between words based on their embeddings.

In [ ]:
import os
import numpy as np
import gensim
from gensim.test.utils import datapath
from sklearn.decomposition import PCA


Download the pre-trained word embeddings from [Google News dataset](https://code.google.com/archive/p/word2vec/) and use the *Gensim* package to load and work with the embeddings.

In [ ]:
# Define the path of the pretrained model
googlenews_vectors = 
gensim_model = gensim.models.KeyedVectors.load_word2vec_format(googlenews_vectors, binary = True)

 Use the following example list of words: 'king', 'queen', 'woman', 'man', 'fish', 'bird', 'snake', 'elephant' (or construct your own list of at least $6$-$10$ meaningful and diverse words.

Reduce the dimensionality of the word embeddings to $2D$ space using PCA and visualize the words in the new latent space.

In [ ]:

def display_pca_scatterplot(model, words=None, sample=0, save=False, file_path='scatterplot.png'):
    


In [ ]:
words_to_explore= [
    'king', 'queen', 'woman', 'man', 'fish', 'bird', 'snake', 'elephant',
]
display_pca_scatterplot(model=gensim_model, words=words_to_explore)


Investigate the relationships between the words based on their embeddings.

In [ ]:
gensim_model.most_similar(positive=["king", "woman"],negative=["man"])


Assess the overall accuracy of the embeddings and analyze strengths and weaknesses across analogy types (syntactic vs. semantic).

In [ ]:
accuracy = gensim_model.evaluate_word_analogies(datapath('questions-words.txt'))

# The first entry stores the overall evaluation score on the entire evaluation set
print(f"Overall evaluation score: {accuracy[0]}")
for current_dict in accuracy[1]:
    correct_count, incorrect_count = len(current_dict['correct']), len(current_dict['incorrect'])
    section_accuracy = correct_count / float(correct_count + incorrect_count)
    print(
        f"Section: {current_dict['section']} - Correct: {correct_count} - Incorrect: {incorrect_count} - Accuracy: {section_accuracy}"
    )

**Question 3.** In this exercise, we will implement the **SkipGram** model of the **Word2Vec** framework.

In [ ]:
# Import the necessary packages
import torch
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm

Prepare the training data

In [ ]:
corpus =  # Define the corpus (preprocessed summaries)


In [ ]:
min_count = 5 # Discard the words that appear less or equal to 'min_count'
window_size =  # You can choose small values to reduce computational cost, but it might affect the performance of the model

# Define the vocabulary, i.e. set of all distinct words that appear more than 'min_count' times in the corpus.
vocab = 
vocab_size = len(vocab) 
# Construct the dictionaries that maps words to unique ids and vice versa.
word2Id = {word: id for id, word in enumerate(vocab)}
id2Word = {id: word for word, id in word2Id.items()}



In [ ]:
# Generate skip-gram training (source, context) word pairs.

# You can implement the following function to generate the source-context word pairs.
# If you wish, you can also modify the argument list
def generate_center_context_pairs(sentences, window_size, word2Id, vocab):
    '''
    Given a list of sentences and a window size, generate a pair of context pairs
    Note that a pair must be discarded if its center or context word appears less than 'min_count'.
    :param sentences: a list of lists of words
    :param window_size: window size
    :param word2Id: a dictionary that maps words to ids
    :param vocab: a set of unique words
    :return: a list of centerId-contextId word pairs
    '''
    data = []

    return data

pairs = generate_center_context_pairs(sentences=sentences, window_size=window_size, word2Id=word2Id, vocab=vocab)
# Convert the list to a torch tensor
pairs = torch.as_tensor(pairs, dtype=torch.long)

In [ ]:
# You can examine the training data
print(f"Vocab size: {vocab_size}")
print(f"Number of center-context pairs: {pairs.shape[0]}")

Implement the SkipGram Model with Softmax Function

In [ ]:
# Define the Skip-gram Model with Softmax
class SkipGramModel(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(SkipGramModel, self).__init__()
        # Complete the implementation
        
    def forward(self, center_word):
    
    def get_center_embs(self, center_words = None):
    
    def get_context_embs(self, context_words):
        

Set the parameters required for training the model such as learning rate, embedding dimension, epochs and batch size.
Note that chosen values for the parameters might/might not affect the optimization and the training time.

In [ ]:
lr = 
epochs_num = 
embedding_dim = 100
batch_size = 

# Initialize model, loss, and optimizer
model = SkipGramModel(vocab_size, embedding_dim)
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

# Set the platform that we will use for training.
device = torch.device('mps') #torch.device("cuda" if torch.cuda.is_available() else "cpu") (for Mac M1/M2, torch.device('mps'))
model.to(device)

Train the model

In [ ]:
# Build a data loader for pairs vector in pytorch
data_loader = DataLoader(pairs, batch_size=batch_size, shuffle=True)

# Training loop
for epoch in tqdm(range(epochs_num)):
    total_loss = 0
    for batch_pairs in tqdm(data_loader, desc=f"Epoch {epoch} - Current batch progress"):
        # Transfer the batch data to the memory of the 'device' (i.e. gpu if used)
        batch_pairs = batch_pairs.to(device)
        # Define the center and context words
        center_word, context_word = batch_pairs[:, 0], batch_pairs[:, 1]
        
        # Forward pass
        optimizer.zero_grad()
        output = model(center_word)
        
        # Calculate loss
        loss = loss_function(output, context_word)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    print(f"Epoch {epoch+1}, Loss: {total_loss/len(pairs):.4f}")  
        
# Transfer the model back to the cpu.
model.to('cpu')

Generate Word Embeddings and Visualize

In [ ]:
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import seaborn as sns

words_to_explore= [
    'king', 'queen', 'woman', 'man', 'boy', 'daughter', 
    'dog', 'cat', 'horse', 'elephant', 'fish', 'bird', 'lizard', 'snake',
    #'doctor', 'nurse', 'scientist', 'teacher', 'engineer', 'artist', 'musician', 'writer',
    #'denmark', 'sweden', 'france', 'germany', 'spain', 'italy', 'portugal', 'turkey',
]
selected_ids = torch.tensor([word2Id[word] for word in words_to_explore])
selected_word_embeddings = model.get_center_embs(selected_ids).detach()


# Visualize the word embeddings using PCA for dimensionality reduction to 2D



Evaluate Word Similarity

In [ ]:
names = [id2Word[i] for i in range(len(id2Word))]
word_embs_df = pd.DataFrame(model.get_center_embs().detach(), index=names)

word1, word2 = "king", "queen"
# Test cosine similarity function
print(f"Cosine similarity between '{word1}' and '{word2}': {cosine_similarity_terms(word1, word2, word_embs_df)}")
word = "king"
print(f"Top 5 similar words to '{word}': {top_similar_words(word, word_embs_df, names, top_n=5)}")

